We'll use both _pu_ ('official') and _ku suli_ ('unofficial' but highly recognised) words:

In [ ]:
import requests
import json
URL = "https://lipu-linku.github.io/jasima/data.json"
data = json.loads(requests.get(URL).text)['data']
words = [word for word, info in data.items()
    if info['book'] in ('pu', 'ku suli')]

Let's remove all the words that don't fit into our 17-char pangram.

In [ ]:
CONSONANTS = 'ptksmnljw'
VOWELS = 'aeiou'
wp = words_pangram = [w for w in words if all((
    all(not w.startswith(v) for v in VOWELS),      # can't start with a vowel
    all(not 'n'+v in w      for v in VOWELS),      # `n` should only end a syllable
    all(w.count(c) <= 1     for c in CONSONANTS),  # no repeated consonants
))]

We'll use a set-based algorithm by looking at the consonants used. Let's keep track of those, shall we?

In [ ]:
wp2 = [(w, set(w) - set('aiueo')) for w in wp]

Okay! Now to evaluate. The generator below effectively jumps down a tree of all Toki Pona sentences from the words above, albeit only to a depth where the sentence is 17 letters, and skipping any words which 

In [ ]:
from typing import List

LETTERS = set(VOWELS + CONSONANTS)
PANGRAM_LENGTH = 17
ALPHABETICAL = False

def pangrams(
    words: List[str] = [],
    seen_consonants: set = set()
):
    done = False
    all_letters = ''.join(words)
    l = len(all_letters)
    if l > PANGRAM_LENGTH:
        return
    if l == PANGRAM_LENGTH:
        if set(all_letters) == LETTERS:
            yield words
    else:
        i = 0
        if ALPHABETICAL and words:
            i = wp.index(words[-1])
        for w, consonants in wp2[i:]:
            if not seen_consonants & consonants:
                yield from pangrams(words + [w], seen_consonants | consonants)

The `ALPHABETICAL` constant trims down processing time by only looking at sentences which are in alphabetical order.

In [ ]:
for i in pangrams():
    print(' '.join(i))

For the results of `ALPHABETICAL = True`, see the attached `.txt` file. If you want _every_ pangram – or you want _ku lili_ words or make 18-letter pangrams – hopefully this code isn't too much to modify!